In [10]:
# Import required libraries
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')



In [13]:
# Define file paths - adjust this path to match your project location
# Replace this with the actual path to your work_supa project
project_path = Path(r"C:\Users\Eddy\YTM Capital Dropbox\Eddy Winiarz\Trading\COF\Models\Unfinished Models\Eddy\Python Projects\work_supa")

parquet_files = {
    'universe': project_path / 'universe' / 'universe.parquet',
    'portfolio': project_path / 'portfolio' / 'portfolio.parquet',
    'combined_runs': project_path / 'runs' / 'combined_runs.parquet',
    'run_monitor': project_path / 'runs' / 'run_monitor.parquet',
    'bond_z': project_path / 'historical g spread' / 'bond_z.parquet'
}

print("📊 Pipeline Parquet Files to DataFrames")
print("=" * 50)
print("🔍 Checking parquet file availability:")
for name, path in parquet_files.items():
    status = "✅" if path.exists() else "❌"
    print(f"{status} {name}: {path}")
print()

# Load all parquet files into DataFrames
print("📥 Loading parquet files into DataFrames...")
print("=" * 50)

# Dictionary to store all DataFrames
dataframes = {}

for name, file_path in parquet_files.items():
    if file_path.exists():
        try:
            # Load the parquet file
            df = pd.read_parquet(file_path)
            
            # Store in dictionary
            dataframes[name] = df
            
            # Also create a variable with the same name as the file
            globals()[name] = df
            
            print(f"✅ {name}: {df.shape[0]:,} rows × {df.shape[1]} columns ({file_path.stat().st_size / 1024 / 1024:.1f} MB)")
            
        except Exception as e:
            print(f"❌ Error loading {name}: {e}")
    else:
        print(f"❌ File not found: {file_path}")

print(f"\n📊 Total DataFrames loaded: {len(dataframes)}")

# Simple .info() display for all DataFrames
print("📊 DATAFRAME INFO SUMMARY")
print("=" * 80)


# Simple .info() for all DataFrames
print("📊 DATAFRAME INFO")
print("=" * 80)

for name, df in dataframes.items():
    print(f"\n {name.upper()}")
    print("=" * 40)
    df.info()
    print()

📊 Pipeline Parquet Files to DataFrames
🔍 Checking parquet file availability:
✅ universe: C:\Users\Eddy\YTM Capital Dropbox\Eddy Winiarz\Trading\COF\Models\Unfinished Models\Eddy\Python Projects\work_supa\universe\universe.parquet
✅ portfolio: C:\Users\Eddy\YTM Capital Dropbox\Eddy Winiarz\Trading\COF\Models\Unfinished Models\Eddy\Python Projects\work_supa\portfolio\portfolio.parquet
✅ combined_runs: C:\Users\Eddy\YTM Capital Dropbox\Eddy Winiarz\Trading\COF\Models\Unfinished Models\Eddy\Python Projects\work_supa\runs\combined_runs.parquet
✅ run_monitor: C:\Users\Eddy\YTM Capital Dropbox\Eddy Winiarz\Trading\COF\Models\Unfinished Models\Eddy\Python Projects\work_supa\runs\run_monitor.parquet
✅ bond_z: C:\Users\Eddy\YTM Capital Dropbox\Eddy Winiarz\Trading\COF\Models\Unfinished Models\Eddy\Python Projects\work_supa\historical g spread\bond_z.parquet

📥 Loading parquet files into DataFrames...
✅ universe: 38,917 rows × 47 columns (5.1 MB)
✅ portfolio: 9,390 rows × 51 columns (1.2 MB)
✅ co